In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import os, codecs, string, random
from numpy.random import seed as random_seed
from numpy.random import shuffle as random_shuffle

%load_ext autoreload
%autoreload 2
%matplotlib inline

import re
from tqdm import tqdm

import warnings; warnings.simplefilter('ignore')

import spacy, nltk, gensim
import pyLDAvis.gensim

import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

In [3]:
seed = 1
random.seed(seed)
dirnames = "data/"+os.listdir("data")[0]
dirnames += "/"+ os.listdir(dirnames)[0]+"/"

In [89]:
def get_directories():
    dirnames = "data/"+os.listdir("data")[0]
    dirnames += "/" + os.listdir(dirnames)[0]+"/"
    return dirnames, os.listdir(dirnames)

In [90]:
dirpath, dirnames = get_directories()

In [2]:
from tqdm import tqdm, tqdm_notebook

In [38]:
def save_data(data, filename):
    savename = "./"+filename+".pkl"
    data.to_pickle(savename)
    return savename

In [5]:
def load_data(savename):
    data = pd.read_pickle(savename)
    return data

In [36]:
def remove_file(savename):
    os.remove(savename)

In [120]:
def read_file_quick(data, filename, dirname, dirpath):
    
    with open(dirpath + dirname + "/" + filename) as current_file:
        doc = current_file.read()
        
        main_label = dirname
        labels = dirname.split(".")
        file_id = current_file.name[-5:]
        text = "".join(re.split("\nLines: [0123456789]+\n", doc)[1:])
        
        data = data.append(pd.DataFrame(np.array([[main_label, labels, file_id, text]]),\
                                        columns=["main_label","labels","file_id", "text"]))
    return data

In [132]:
def read_file(data, filename, dirname, dirpath):
    with open(dirpath + dirname + "/" + filename) as current_file:
        doc = current_file.read()
        
        main_label = dirname
        labels = dirname.split(".")
        file_id = current_file.name[-5:]
        
        newsgroups, subject, organization, date, n_lines = "","","","",""
        if(re.search("Newsgroups: .*\n", doc) and re.findall("Newsgroups: .*\n", doc)[0]):
            newsgroups = re.findall("Newsgroups: .*\n", doc)[0]\
                            .split("\n")[0].split("Newsgroups: ")[-1].split(",")

        if(re.search("Subject: .*\n", doc) and re.findall("Subject: .*\n", doc)[0]):
            subject = re.findall("Subject: .*\n", doc)[0]\
                            .split("\n")[0].split("Subject: ")[-1]

        if(re.search("Organization: .*\n", doc) and re.findall("Organization: .*\n", doc)[0]):
            organization = re.findall("Organization: .*\n", doc)[0]\
                            .split("\n")[0].split("Organization: ")[-1]

        if(re.search("Date: .*\n", doc) and re.findall("Date: .*\n", doc)[0]):
            date = re.findall("Date: .*\n", doc)[0]\
                            .split("\n")[0].split("Date: ")[-1]

        if(re.search("Lines: .*\n", doc) and re.findall("Lines: .*\n", doc)[0]):
            n_lines = re.findall("Lines: .*\n", doc)[0]\
                            .split("\n")[0].split("Lines: ")[-1]
        
        text = "".join(re.split("\nLines: [0123456789]+\n", doc)[1:])
        
        data = data.append(pd.DataFrame(np.array([[main_label, labels, file_id, newsgroups, subject, organization, date, n_lines, text]]),\
                                        columns=["main_label","labels","file_id","newsgroups","subject","organization","date","n_lines","text"]))
    return data

In [161]:
def read_directory_quick(dirnames, dirpath, data):
    for filename in tqdm(os.listdir(dirpath+dirname)): 
        data = read_file_quick(data, filename, dirname, dirpath)
    return data

In [ ]:
def read_directory(dirnames, dirpath, data):
    for filename in tqdm(os.listdir(dirpath+dirname)):  
        data = read_file(data, filename, dirname, dirpath)
    return data

In [154]:
# data = pd.DataFrame(columns = ["main_label", "labels","file_id","newsgroups","subject","organization","date","n_lines","text"])
# for dirname in tqdm_notebook(dirnames):
#     data = read_directory(dirnames, dirpath, data)


In [159]:
data = pd.DataFrame(columns = ["main_label","labels","file_id","text"])
for dirname in tqdm_notebook(dirnames):
    data = read_directory_quick(dirnames, dirpath, data)

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1085.04it/s]


(1000, 4)


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1084.17it/s]


(2000, 4)


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1035.19it/s]


...

(17997, 4)


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 567.02it/s]


(18997, 4)


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 518.03it/s]


(19997, 4)


In [7]:
data.shape

(19997, 4)

In [162]:
savename = save_data(data, "quick_data_1")

In [163]:
savename

'./quick_data_1.pkl'

In [6]:
data = load_data("quick_data_1.pkl")

In [8]:
temp = data.copy()
temp= temp.iloc[12000:12010]
temp.tail(1)

,main_label,labels,file_id,text
0,sci.electronics,"[sci, electronics]",52723,\ng92m3062@alpha.ru.ac.za (Brad Meier) writes:...


In [9]:
from collections import Counter
from string import punctuation
from spacy.lang.en.stop_words import STOP_WORDS

In [10]:
nlp = spacy.load("en_core_web_sm")

In [167]:
EXCLUDE_CHARS = list(set(punctuation).union(set('’')))

In [168]:
import time

In [181]:
# remove stopwords and then lemmatize
def lemmatizer(doc):
    words = [token.lemma_ for token in nlp(doc) if token.is_stop != True \
             and token.is_punct != True \
             and token.is_space != True \
             and token.is_digit != True \
             and re.search("\\"+"|\\".join(EXCLUDE_CHARS), token.lemma_) is None \
             and re.search("|".join("0123456789"), token.lemma_) is None]
    return words

def lemmatizer_continuous(doc):
    words = [token.lemma_ for token in nlp(doc) if token.is_stop != True \
             and token.is_punct != True \
             and token.is_space != True \
             and token.is_digit != True \
             and re.search("\\"+"|\\".join(EXCLUDE_CHARS), token.lemma_) is None \
             and re.search("|".join("0123456789"), token.lemma_) is None]
    return " ".join(words)

In [179]:
# from tqdm._tqdm_notebook import tqdm_notebook
# tqdm_notebook.pandas()
# need to change pandas version to be able to import and use the progress_apply functionality

In [182]:
t1 = time.time()
data["continuous"] = data.text.apply(lambda x: lemmatizer_continuous(x))
t2 = time.time()
data["words"] = data.text.apply(lambda x: lemmatizer(x))
t3 = time.time()

lemmatizer_continuous: 1622.25 seconds for 10 lines
lemmatizer: 1606.49 seconds for 10 lines


In [189]:
print("lemmatizer_continuous: {:.2f} seconds  = {:.2f} minutes for {} lines".format(t2-t1, (t2-t1)/60, len(data)))
print("lemmatizer: {:.2f} seconds = {:.2f} minutes for {} lines".format(t3-t2, (t3-t2)/60, len(data)))


lemmatizer_continuous: 1622.25 seconds  = 27.04 minutes for 19997 lines
lemmatizer: 1606.49 seconds = 26.77 minutes for 19997 lines


In [190]:
save_data(data, "quick_data_processed_1")

'./quick_data_processed_1.pkl'

In [11]:
data_processed = load_data("quick_data_processed_1.pkl")

In [12]:
data_processed.shape

(19997, 6)

In [58]:
bag_of_words = Counter(data_processed.words.explode())

In [15]:
bag_of_words.most_common(10)

[('be', 29407),
 ('not', 29401),
 ('the', 26935),
 ('write', 18496),
 ('in', 18098),
 ('article', 12943),
 ('know', 11543),
 ('good', 10416),
 ('like', 10379),
 ('people', 10328)]

In [ ]:
# BOOOOh should remove some more stopwords. very common list is not specific. 
# should then use Luhn: remove most common and least commmon

In [27]:
# should also have the dirname where it comes form
# should check that we have roughly the same amount of datapoints in each class

In [16]:
import nltk

In [17]:
from nltk.corpus import stopwords

In [18]:
stopWords = set(stopwords.words('english'))
# join with other set and .unique()
# also should remove numbers that are within words

In [19]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


In [71]:
from itertools import dropwhile, islice, count


In [63]:
newbag  = bag_of_words.copy()

In [67]:
# for key, count in islice(bag_of_words.most_common(),10):
#     print(key, count)

In [122]:
# percentage min is the percentage of words that are less frequent that you want to drop e.g. 10% less frequent
# sorted from most frequent to least frequent:
# so if you take most_common()[minimum value - maximum value]
# you need to have minimum value = the most frequent you want = the maximum percentage. but you need to inverse the percentages
# because if you want 90% less frequent words, and it is sorted by most frequent, you need to discard the top 10%
# similarly if you want to remove the 10% less frequent you need to remove the slice from 90% to 100%
# so use a nice 1-probability
def remove_between_thresholds(bag_of_words, percentage_min, percentage_max):
    t_min = int(len(bag_of_words)*(1-percentage_min/100))
    t_max = int(len(bag_of_words)*(1-percentage_max/100))
    return Counter(dict(bag_of_words.most_common()[t_max: t_min]))

In [108]:
# these are based on the count....
def remove_less_frequent(bag_of_words, threshold):
    for key, count in dropwhile(lambda x: x[1] > threshold, bag_of_words.most_common()):
        del bag_of_words[key]
    return bag_of_words

def remove_most_frequent(bag_of_words, threshold):
    for key, count in takewhile(lambda x: x[1] < threshold, bag_of_words.most_common()):
        del bag_of_words[key]
    return bag_of_words

In [124]:
bag_of_words_slim = remove_between_thresholds(bag_of_words, 5, 95)

In [136]:
# le = preprocessing.LabelEncoder()

In [134]:
# le.fit(data.labels.explode().unique())

In [135]:
# at= temp.iloc[12000:12010]
# at.head(1)

In [132]:
# le.classes_

In [175]:
# def one_hot_labels(data, le):
#     data["one_hot_labels"] = data.apply(lambda x: le.transform(x.labels), axis = 1)
#     return data
# pandas.get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False, dtype=None) → 'DataFrame'

In [137]:
# new.a.apply(lambda x: pd.get_dummies(x))

In [138]:
# le.transform(at.iloc[0,0])

In [139]:
# just a test
# le.inverse_transform(le.transform(["sci", "electronics"]))

In [140]:
# new = le.transform(at.labels.explode())

In [226]:
data_processed.head(1)

,main_label,labels,file_id,text,continuous,words
0,alt.atheism,"[alt, atheism]",49960,\nArchive-name: atheism/resources\nAlt-atheism...,archive atheism resource alt atheism archive r...,"[archive, atheism, resource, alt, atheism, arc..."


In [230]:
X = data_processed.continuous
y = data_processed.main_label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 1)

In [231]:
mlb = MultiLabelBinarizer()
tfidf = TfidfVectorizer()
mlb.fit(y_train)
tfidf.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [233]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [234]:
from sklearn.metrics import accuracy_score

In [238]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer


In [249]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(17997, 71038)

In [250]:
tfidf = TfidfTransformer()
X_train_tfidf = tfidf.fit_transform(X_train_counts)
X_train_tfidf.shape

(17997, 71038)

In [244]:
y_train.shape

(17997,)

In [254]:
mnb =  MultinomialNB()
mnb.fit(X_train_tfidf, y_train)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [255]:
X_test_count = count_vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_count)

In [256]:
mnb_predictions= mnb.predict(X_test_tfidf)
print(accuracy_score(y_test, mnb_predictions))


0.8215


In [259]:
rfc = RandomForestClassifier(max_depth = None, n_estimators = 150, random_state = 0)
rfc.fit(X_train_tfidf, mlb.transform(y_train))
rfc_predictions = rfc.predict(X_test_tfidf)
print(accuracy_score(mlb.transform(y_test), rfc_predictions))

        

0.333


In [ ]:
# so we know how to predict the "main" label but its consistently wrong...
# now on to the other issues: predict labels individually
# shoudl also look at different methods
# can try to use only the count vectorizer
# can try to predict mulitiple labels
# can do sentiment analysis
# should try other classifiers